<a href="https://colab.research.google.com/github/sarang-manohar/Coursera_Capstone/blob/main/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook puts in the seed of the Capstone Project with Coursera!

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder
import folium
from pandas.io.json import json_normalize
print('Libraries imported')

Libraries imported


In [2]:
html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(html_data.text, 'html.parser')
table = soup.find("table",{"rules":"all"})

table_data = []

for row in table.find_all('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_data.append(cell)

df=pd.DataFrame(table_data)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df['Address'] = df['Borough']+', Toronto, Ontario, Canada, '+df['PostalCode']
df.head()

,PostalCode,Borough,Neighborhood,Address
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A"
1,M4A,North York,Victoria Village,"North York, Toronto, Ontario, Canada, M4A"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Downtown Toronto, Toronto, Ontario, Canada, M5A"
3,M6A,North York,"Lawrence Manor, Lawrence Heights","North York, Toronto, Ontario, Canada, M6A"
4,M7A,Queen's Park,Ontario Provincial Government,"Queen's Park, Toronto, Ontario, Canada, M7A"


In [3]:
lat=[]
long=[]
for i, pc in enumerate(df['PostalCode']):
    with requests.Session() as session:
        g = geocoder.arcgis('{}, Toronto, Canada'.format(pc),session=session)
    lat.append(g.latlng[0])
    long.append(g.latlng[1])

df['Latitude'] = lat
df['Longitude'] = long

df.head()

,PostalCode,Borough,Neighborhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A",43.75245,-79.32991
1,M4A,North York,Victoria Village,"North York, Toronto, Ontario, Canada, M4A",43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Downtown Toronto, Toronto, Ontario, Canada, M5A",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights","North York, Toronto, Ontario, Canada, M6A",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,"Queen's Park, Toronto, Ontario, Canada, M7A",43.66253,-79.39188


In [4]:
address = 'Toronto, Ontario, Canada'

g = geocoder.arcgis(address)
latitude = g.latlng[0]
longitude = g.latlng[1]
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.648690000000045, -79.38543999999996.


In [5]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto) 

map_Toronto

In [6]:
CLIENT_ID = 'Q5LJU1V0DPCB0R5XDREYLILRZGTSPSBFPBX1QGAQHZAAPNPK' # your Foursquare ID
CLIENT_SECRET = 'U4XJXABOBZBSHOWMLEQYHNEYVSXAR0VBAYAUGHLBBQ3LJIVX' # your Foursquare Secret
VERSION = '20210704' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
ACCESS_TOKEN = '0HUXGDI5F3EDNXPW3RW21KJNV4TB3Z4ITMA34M5VRKXR0HRA'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q5LJU1V0DPCB0R5XDREYLILRZGTSPSBFPBX1QGAQHZAAPNPK
CLIENT_SECRET:U4XJXABOBZBSHOWMLEQYHNEYVSXAR0VBAYAUGHLBBQ3LJIVX


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&GET'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION)
results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.drop({'categories'},axis=1,inplace=True)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

<ipython-input-13-54553548249f>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


In [15]:
df.head(1)

,PostalCode,Borough,Neighborhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A",43.75245,-79.32991


In [41]:
df_test = df.copy()
df_test = df_test.head(2)
df_test

,PostalCode,Borough,Neighborhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A",43.75245,-79.32991
1,M4A,North York,Victoria Village,"North York, Toronto, Ontario, Canada, M4A",43.73057,-79.31306


In [110]:
venues_list=[]
for name, lat, lng in zip(df['Neighborhood'], df['Latitude'], df['Longitude']):
    print(name)
        
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        500, 
        LIMIT)
        
    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    # return only relevant information for each nearby venue
    venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    # return only relevant information for each nearby venue
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
              'Neighborhood Latitude', 
              'Neighborhood Longitude', 
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']

nearby_venues.sample(10)


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
478,Central Bay Street,43.65609,-79.38493,Japango,43.655268,-79.385165,Sushi Restaurant
1956,Enclave of M5E,43.64869,-79.38544,Royal Alexandra Theatre,43.647255,-79.387530,Theater
1598,Agincourt,43.79452,-79.26708,Canton Kitchen 廣東小炒,43.797772,-79.270981,Chinese Restaurant
125,"Garden District, Ryerson",43.65739,-79.37804,Jazz Bistro,43.655678,-79.379276,Music Venue
424,Berczy Park,43.64536,-79.37306,Garrison Bespoke,43.648102,-79.376334,Tailor Shop
614,"Richmond, Adelaide, King",43.64970,-79.38258,GoodLife Fitness Toronto 137 Yonge Street,43.651242,-79.378068,Gym
62,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,AllSaints,43.724378,-79.453646,Clothing Store
697,"Fairview, Henry Farm, Oriole",43.78097,-79.34781,McDonald's,43.778407,-79.343574,Fast Food Restaurant
607,"Richmond, Adelaide, King",43.64970,-79.38258,Tim Hortons,43.646862,-79.382544,Coffee Shop
803,"Little Portugal, Trinity",43.64848,-79.41774,Pizzeria Libretto,43.648979,-79.420604,Pizza Place


In [111]:
nearby_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,15,15,15,15,15,15
"Alderwood, Long Branch",4,4,4,4,4,4
"Bathurst Manor, Wilson Heights, Downsview North",2,2,2,2,2,2
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",21,21,21,21,21,21
Woburn,5,5,5,5,5,5
Woodbine Heights,18,18,18,18,18,18


In [113]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue Category'].unique())))

There are 264 uniques categories.


In [114]:
# one hot encoding
toronto_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [115]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
98,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
99,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
100,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [116]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0   Chinese Restaurant  0.13
1     Department Store  0.07
2         Skating Rink  0.07
3  Shanghai Restaurant  0.07
4     Sushi Restaurant  0.07


----Alderwood, Long Branch----
                   venue  freq
0      Convenience Store  0.25
1                    Pub  0.25
2  Performing Arts Venue  0.25
3                    Gym  0.25
4            Zoo Exhibit  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0               Home Service   0.5
1                Men's Store   0.5
2         Mexican Restaurant   0.0
3  Middle Eastern Restaurant   0.0
4         Miscellaneous Shop   0.0


----Bayview Village----
                        venue  freq
0                       Trail  0.50
1  Construction & Landscaping  0.25
2                        Park  0.25
3                 Zoo Exhibit  0.00
4               Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      

In [117]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [118]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Department Store,Skating Rink,Shanghai Restaurant,Sushi Restaurant,Supermarket,Grocery Store,Bakery,Badminton Court,Shopping Mall
1,"Alderwood, Long Branch",Convenience Store,Pub,Performing Arts Venue,Gym,Zoo Exhibit,Movie Theater,Noodle House,Nightclub,New American Restaurant,Music Venue
2,"Bathurst Manor, Wilson Heights, Downsview North",Home Service,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater
3,Bayview Village,Trail,Construction & Landscaping,Park,Zoo Exhibit,Movie Theater,Nightclub,New American Restaurant,Music Venue,Museum,Moroccan Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Butcher,Sports Club,Liquor Store,Fast Food Restaurant,Thai Restaurant,Restaurant,Comfort Food Restaurant


In [119]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 4, 1, 1, 1, 1, 1, 1])

In [120]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A",43.75245,-79.32991,4.0,Food & Drink Shop,Fast Food Restaurant,Park,Nightclub,New American Restaurant,Music Venue,Museum,Movie Theater,Moroccan Restaurant,Office
1,M4A,North York,Victoria Village,"North York, Toronto, Ontario, Canada, M4A",43.73057,-79.31306,4.0,Park,Grocery Store,Zoo Exhibit,Movie Theater,Nightclub,New American Restaurant,Music Venue,Museum,Moroccan Restaurant,Office
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Downtown Toronto, Toronto, Ontario, Canada, M5A",43.65512,-79.36264,1.0,Coffee Shop,Breakfast Spot,Yoga Studio,Gym / Fitness Center,Food Truck,Sushi Restaurant,Thai Restaurant,Theater,Bakery,Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights","North York, Toronto, Ontario, Canada, M6A",43.72327,-79.45042,1.0,Clothing Store,Furniture / Home Store,Toy / Game Store,Women's Store,American Restaurant,Food Court,Bookstore,Men's Store,Cosmetics Shop,Platform
4,M7A,Queen's Park,Ontario Provincial Government,"Queen's Park, Toronto, Ontario, Canada, M7A",43.66253,-79.39188,1.0,Coffee Shop,Sandwich Place,Italian Restaurant,Bank,Theater,Gastropub,Café,Park,Mediterranean Restaurant,Fried Chicken Joint


In [130]:
toronto_merged.shape

(102, 17)

In [133]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [134]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,Etobicoke,-79.51113,0.0,Pool,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater


In [135]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,-79.36264,1.0,Coffee Shop,Breakfast Spot,Yoga Studio,Gym / Fitness Center,Food Truck,Sushi Restaurant,Thai Restaurant,Theater,Bakery,Restaurant
3,North York,-79.45042,1.0,Clothing Store,Furniture / Home Store,Toy / Game Store,Women's Store,American Restaurant,Food Court,Bookstore,Men's Store,Cosmetics Shop,Platform
4,Queen's Park,-79.39188,1.0,Coffee Shop,Sandwich Place,Italian Restaurant,Bank,Theater,Gastropub,Café,Park,Mediterranean Restaurant,Fried Chicken Joint
5,Etobicoke,-79.52831,1.0,Pharmacy,Bank,Park,Shopping Mall,Skating Rink,Grocery Store,Café,Monument / Landmark,Moroccan Restaurant,Noodle House
6,Scarborough,-79.19662,1.0,Zoo Exhibit,Fast Food Restaurant,Office,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,-79.38146,1.0,Coffee Shop,Hotel,Café,Restaurant,Gym,Japanese Restaurant,Deli / Bodega,Asian Restaurant,Concert Hall,Bank
99,Downtown Toronto,-79.38133,1.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Café,Gay Bar,Men's Store,Fast Food Restaurant,Dance Studio,Pub
100,East Toronto Business,-79.38544,1.0,Coffee Shop,Hotel,Café,Bar,Sushi Restaurant,Asian Restaurant,Restaurant,Italian Restaurant,Taco Place,Concert Hall
101,Etobicoke,-79.48945,1.0,Sushi Restaurant,Chinese Restaurant,Bank,Flower Shop,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Park,Zoo Exhibit,Nightclub


In [136]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,North York,-79.44847,2.0,Home Service,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater
53,North York,-79.50178,2.0,Home Service,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater
62,Central Toronto,-79.41848,2.0,Home Service,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater


In [137]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,-79.15875,3.0,Bar,Zoo Exhibit,Office,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant


In [138]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,-79.32991,4.0,Food & Drink Shop,Fast Food Restaurant,Park,Nightclub,New American Restaurant,Music Venue,Museum,Movie Theater,Moroccan Restaurant,Office
1,North York,-79.31306,4.0,Park,Grocery Store,Zoo Exhibit,Movie Theater,Nightclub,New American Restaurant,Music Venue,Museum,Moroccan Restaurant,Office
16,York,-79.43036,4.0,Park,Tennis Court,Hockey Arena,Grocery Store,Field,Trail,Moroccan Restaurant,New American Restaurant,Music Venue,Museum
18,Scarborough,-79.17470,4.0,Gym / Fitness Center,Construction & Landscaping,Park,Movie Theater,Noodle House,Nightclub,New American Restaurant,Music Venue,Museum,Zoo Exhibit
27,North York,-79.35558,4.0,Residential Building (Apartment / Condo),Park,Moroccan Restaurant,Nightclub,New American Restaurant,Music Venue,Museum,Movie Theater,Zoo Exhibit,Office
32,Scarborough,-79.23117,4.0,Park,Indian Restaurant,Grocery Store,Zoo Exhibit,Movie Theater,Nightclub,New American Restaurant,Music Venue,Museum,Moroccan Restaurant
35,East York/East Toronto,-79.33418,4.0,Fabric Shop,Park,Film Studio,Intersection,Zoo Exhibit,Museum,Nightclub,New American Restaurant,Music Venue,Moroccan Restaurant
39,North York,-79.38060,4.0,Trail,Construction & Landscaping,Park,Zoo Exhibit,Movie Theater,Nightclub,New American Restaurant,Music Venue,Museum,Moroccan Restaurant
45,North York,-79.38060,4.0,Park,Music Venue,Zoo Exhibit,Office,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark
49,North York,-79.48874,4.0,Bakery,Trail,Park,Basketball Court,Zoo Exhibit,Museum,Noodle House,Nightclub,New American Restaurant,Music Venue
